<a href="https://colab.research.google.com/github/Krisztiaan/research-main-xr/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAIN-XR-MD Phase-0 · Colab Launcher
Use this notebook to spin up a Google Colab session with GPU support, install CUDA-enabled JAX, and run a short validation rollout for the project.

## 1. Select GPU Runtime
Before running any cells below, open **Runtime → Change runtime type → GPU**. If you only need a CPU smoke test, you can keep the default CPU runtime and set `JAX_PLATFORMS=cpu` in later cells.

In [1]:
!nvidia-smi || echo 'No NVIDIA GPU attached; fallback to CPU runtime.'

Mon Oct  6 14:39:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Install Dependencies
Installs CUDA-enabled JAX and this repository from Git (change the URL if you are working from a fork).

In [2]:
%%bash
set -euxo pipefail
python -m pip install --upgrade pip setuptools wheel
python -m pip install --upgrade "jax[cuda12_pip]>=0.4.30" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
python -m pip install --upgrade git+https://github.com/krisztiaan/main_xr_md_jax_phase0.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 81.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 66.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 MB 65.7 MB/s  0:00:01
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.5.3
    Uninstalling jaxlib-0.5.3:
      Successfully uninstalled jaxlib-0.5.3
  Attempting uninstall: jax
    Found existing installation: jax 0.5.3
    Uninstalling jax-0.5.3:
      Successfully uninstalled jax-0.5.3

  Cl

+ python -m pip install --upgrade pip setuptools wheel
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
+ python -m pip install --upgrade 'jax[cuda12_pip]>=0.4.30' -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
+ python -m pip install --upgrade git+https://github.com/krisztiaan/main_xr_md_jax_phase0.git
  Running command git clone --filter=blob:none --quiet https://github.com/krisztiaan/main_xr_md_jax_phase0.git /tmp/pip-req-build-jqvfxy09
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/krisztiaan/main_xr_md_jax_phase0.git /tmp/pip-req-build-jqvfxy09 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error o

CalledProcessError: Command 'b'set -euxo pipefail\npython -m pip install --upgrade pip setuptools wheel\npython -m pip install --upgrade "jax[cuda12_pip]>=0.4.30" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html\npython -m pip install --upgrade git+https://github.com/krisztiaan/main_xr_md_jax_phase0.git\n'' returned non-zero exit status 1.

## 3. Clone Repo Snapshot (Editable)
Colab installs the library above; clone if you want to edit code inline.

In [ ]:
%%bash
set -euxo pipefail
if [ ! -d main_xr_md_jax_phase0 ]; then
  git clone https://github.com/krisztiaan/main_xr_md_jax_phase0.git
fi

## 4. Quick GPU Rollout
Runs a short training loop to verify everything is wired correctly. Adjust hyperparameters for longer jobs.

In [ ]:
%%bash
set -euxo pipefail
cd main_xr_md_jax_phase0
python -m pip install -r requirements.txt
python -m mxrmd_jax.train_jax \
  --env craftax \
  --env-id craftax-classic-v1 \
  --num-envs 256 \
  --unroll 16 \
  --total-frames 262144 \
  --r-reset 20 \
  --run-dir /content/runs/colab_demo
ls -R /content/runs/colab_demo

## 5. Save Artifacts
Mount Google Drive if you plan to keep checkpoints beyond this session.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!rsync -a /content/runs/colab_demo /content/drive/MyDrive/mxrmd_colab_runs